In [8]:
from collections.abc import Callable

import FinanceDataReader as fdr
from pandas.core.frame import DataFrame

In [2]:
from finance_fns import stock_handle

samsung_code = stock_data.find_code('삼성전자')
samsung_code

'005930'

In [5]:
def get_close_df(stock_code: str, start_date: str, end_date: str = None) -> DataFrame:
    try:
        if end_date is None:
            df = fdr.DataReader(stock_code, start_date)
        else:
            df = fdr.DataReader(stock_code, start_date, end_date)

    except Exception as e:
        print(f"Error: {e}")

    return df[['Close']]

df = get_close_df(samsung_code, '2024-3')
df


<class 'pandas.core.frame.DataFrame'>


,Close
Date,
2024-03-04,74900
2024-03-05,73700
2024-03-06,72900
2024-03-07,72200
2024-03-08,73300
...,...
2024-12-09,53400
2024-12-10,54000
2024-12-11,54000


In [11]:
def add_to_df(df: DataFrame) -> Callable[[str, Callable], DataFrame]:
    df = df.copy()
    def adder(name: str, fn: Callable) -> DataFrame:
        df[name] = fn(df)
        return df
    return adder
df_adder = add_to_df(df)

In [15]:
df_adder('Short', lambda df: df['Close'].ewm(span=9, adjust=False).mean())
df_adder('Long', lambda df: df['Close'].ewm(span=26, adjust=False).mean())
df_adder('MACD', lambda df: df['Short'] - df['Long'])
df_adder('Signal', lambda df: df['MACD'].ewm(span=13, adjust=False).mean())
df_adder('MACD_Oscillator', lambda df: df['MACD'] - df['Signal'])

,Close,Short,Long,MACD,Signal,MACD_Oscillator
Date,,,,,,
2024-03-04,74900,74900.000000,74900.000000,0.000000,0.000000,0.000000
2024-03-05,73700,74660.000000,74811.111111,-151.111111,-21.587302,-129.523810
2024-03-06,72900,74308.000000,74669.547325,-361.547325,-70.153019,-291.394306
2024-03-07,72200,73886.400000,74486.617894,-600.217894,-145.876573,-454.341321
2024-03-08,73300,73769.120000,74398.720272,-629.600272,-214.979958,-414.620314
...,...,...,...,...,...,...
2024-12-09,53400,54178.588318,55576.266172,-1397.677854,-1424.787770,27.109916
2024-12-10,54000,54142.870654,55459.505715,-1316.635060,-1409.337383,92.702323
2024-12-11,54000,54114.296523,55351.394180,-1237.097657,-1384.731708,147.634051


In [18]:
def macd_oscillator(df: DataFrame, short_n: int, long_n: int, signal_n: int) -> DataFrame:
    df_adder = add_to_df(df)
    df_adder('Short', lambda df: df['Close'].ewm(span=short_n, adjust=False).mean())
    df_adder('Long', lambda df: df['Close'].ewm(span=long_n, adjust=False).mean())
    df_adder('MACD', lambda df: df['Short'] - df['Long'])
    df_adder('Signal', lambda df: df['MACD'].ewm(span=signal_n, adjust=False).mean())
    final = df_adder('MACD_Oscillator', lambda df: df['MACD'] - df['Signal'])
    return final[['MACD', 'Signal', 'MACD_Oscillator']]

In [19]:
macd_df = macd_oscillator(df, 9, 26, 13)
macd_df


,MACD,Signal,MACD_Oscillator
Date,,,
2024-03-04,0.000000,0.000000,0.000000
2024-03-05,-151.111111,-21.587302,-129.523810
2024-03-06,-361.547325,-70.153019,-291.394306
2024-03-07,-600.217894,-145.876573,-454.341321
2024-03-08,-629.600272,-214.979958,-414.620314
...,...,...,...
2024-12-09,-1397.677854,-1424.787770,27.109916
2024-12-10,-1316.635060,-1409.337383,92.702323
2024-12-11,-1237.097657,-1384.731708,147.634051
